# DATR-406 Final Project
## Prep ACS Data

The raw ACS data cannot be used as features directly.  This notebook extracts the raw variables from the files at the
county level for use in following steps.  Data was manually downloaded from the links below, unzipped as necessary, and moved to the **data > raw** subfolder of this project.

[Raw data zip](https://www2.census.gov/acs2005_2009_5yr/summaryfile/2005-2009_ACSSF_All_In_2_Giant_Files(Experienced-Users-Only)/All_Geographies_Not_Tracts_Block_Groups.zip)<br>
[Sequence Number](https://www2.census.gov/acs2005_2009_5yr/summaryfile/Sequence_Number_and_Table_Number_Lookup.xls)<br>
[Table shells](https://www2.census.gov/acs2005_2009_5yr/summaryfile/ACS2009_5-Year_TableShells.xls)

In [1]:
import pandas as pd
import re
import os
import zipfile
import shutil
import collections

In [2]:
HEADER_FILE = './data/raw/ACS2009_5-Year_TableShells.xls'
SEQUENCE_FILE = './data/raw/Sequence_Number_and_Table_Number_Lookup.xls'
DATA_PATH = './data/raw/All_Geographies_Not_Tracts_Block_Groups/'

### Step 1: Prep Metadata
The ACS data files do not include headers.  That information in stored in a separate file.  Here, I read and clean the header file.

In [3]:
seq = pd.read_excel(SEQUENCE_FILE)
seq.columns = [x.lower().replace('\n', '_').replace(' ', '_') for x in seq.columns]
seq.drop(['file_id', 'total_cells_in_sequence', 'subject_area'], axis='columns', inplace=True)
seq.rename({'table_title': 'label'}, axis='columns', inplace=True)

seq['table_title'] = ''
seq['universe'] = ''

table_group = seq.groupby('table_id')
for name, group in table_group:
    cur_title = group['label'].values[0]
    cur_universe = re.sub('^Universe: ', '', group['label'].values[1])
    seq.loc[seq['table_id'] == name, 'table_title'] = cur_title
    seq.loc[seq['table_id'] == name, 'universe'] = cur_universe
    
seq = seq.loc[seq['line_number'].notnull(), :]
seq = seq.loc[seq['line_number'] != 0.5, :]
seq.drop(['start_position', 'total_cells_in_table'], axis='columns', inplace=True)

seq['line_number'] = seq['line_number'].astype(int)
seq['variable'] = seq['table_id'] + '_' + seq['line_number'].astype(str).str.zfill(3)

seq.to_csv('data/sequence_info.csv', index=False)

display(seq.head())

,table_id,sequence_number,line_number,label,table_title,universe,variable
2,B07401,1,1,Total living in area 1 year ago:,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY AGE ...,Population 1 year and over in the United States,B07401_001
3,B07401,1,2,1 to 4 years,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY AGE ...,Population 1 year and over in the United States,B07401_002
4,B07401,1,3,5 to 17 years,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY AGE ...,Population 1 year and over in the United States,B07401_003
5,B07401,1,4,18 and 19 years,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY AGE ...,Population 1 year and over in the United States,B07401_004
6,B07401,1,5,20 to 24 years,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY AGE ...,Population 1 year and over in the United States,B07401_005


In [4]:
# grouping by table name to make it easier to find variables of interest
table_meta = seq[['table_title', 'variable']].groupby('table_title').min()
table_meta = table_meta.reset_index()
table_meta['num_groups'] = table_meta['variable'].str.split(' BY ').apply(len)
table_meta.sort_values(['num_groups', 'table_title'], inplace=True)
table_meta.to_csv('data/table_names.csv', index=False)

table_meta.head()

,table_title,variable,num_groups
0,AGE AND NATIVITY OF OWN CHILDREN UNDER 18 YEAR...,B05009_001,1
1,AGE BY LANGUAGE SPOKEN AT HOME BY ABILITY TO S...,B16004_001,1
2,AGE BY LANGUAGE SPOKEN AT HOME FOR THE POPULAT...,B16007_001,1
3,AGE BY LANGUAGE SPOKEN AT HOME FOR THE POPULAT...,B16003_001,1
4,AGE BY RATIO OF INCOME TO POVERTY LEVEL IN THE...,B17024_001,1


The data is stored with logrecno (record number) as the unique identifier.  Each state folder contains a lookup between
logrecno and the components needed to construct geoid.  The header for these lookup files can be found [in this pdf](https://www2.census.gov/acs2005_2009_5yr/summaryfile/ACS_2005-2009_SF_Tech_Doc.pdf), transcribed manually here.

In [5]:
GEO_COLS = ['fileid', 'stusab', 'sumlevel', 'component', 'logrecno', 'us', 'region', 'division', 'statece', 
            'state', 'county', 'cousub', 'place', 'tract', 'blkgrp', 'concit', 'aianhh', 'aianhhfp', 'aihhtli', 
            'aitsce', 'aits', 'anrc', 'cbsa', 'csa', 'metdiv', 'macc', 'memi', 'necta', 'cnecta', 'nectadiv', 
            'ua', 'blank_1', 'cdcurr', 'sldu', 'sldl', 'blank_2', 'blank_3', 'blank_4', 'submcd', 'sdelm', 
            'sdsec', 'sduni', 'ur', 'pci', 'blank_5', 'blank_6', 'puma5', 'blank_7', 'geoid', 'name', 'blank_8']

GEO_WIDTHS = [6, 2, 3, 2, 7, 1, 1, 1, 2, 2, 3, 5, 5, 6, 1, 5, 4, 5, 1, 3, 5, 5, 5, 3, 5, 1, 1, 5, 3, 5, 5, 5, 2, 3,
             3, 6, 3, 5, 5, 5, 5, 5, 1, 1, 6, 5, 5, 5, 40, 200, 50]

print('{} cols, {} widths'.format(len(GEO_COLS), len(GEO_WIDTHS)))

51 cols, 51 widths


### Step 2: Prep Variables to Extract
The list of variables, with labels, are stored in acs_variables.txt.  There are over 20,000 variables in these ACS data.  I have selected a number of demographic variables based on my experience but it by no means an exhaustive list.

In [6]:
acs_vars_raw = pd.read_csv('data/acs_variables.txt', sep='\t')
acs_vars = acs_vars_raw['variable'].tolist()
print('{} ACS variables to download'.format(len(acs_vars)))

202 ACS variables to download


### Step 3: Extract Data by State


In [7]:
# get list of states
states = os.listdir(DATA_PATH)
states = [x.split('_')[0] for x in states if '.zip' in x]
states = [x for x in states if x != 'UnitedStates']
states.sort()
print('{} states, including DC and Puerto Rico'.format(len(states)))

52 states, including DC and Puerto Rico


In [8]:
# create lookup from files to variables
files_to_vars = collections.defaultdict(lambda: collections.defaultdict(list))
for v in acs_vars:
    file_num = seq.loc[seq['variable'] == v, 'sequence_number']
    if file_num.shape[0] > 0:
        file_num = file_num.values[0]
        file_name = 'e20095XX{num}.txt'.format(num='{:07d}'.format(file_num * 1000))
        file_name = file_name.replace('XX', '{st}')
        files_to_vars[file_name]['cols'].append(v)
        files_to_vars[file_name]['seq_num'] = file_num
    else:
        print('variable {} not found'.format(v))
    
# update dict with full list of cols/header for each file
for k in files_to_vars.keys():
    header = seq.loc[seq['sequence_number'] == files_to_vars[k]['seq_num'], 'variable'].values.tolist()
    files_to_vars[k]['header'] = header

In [9]:
# load raw data by state
std_cols = ['ignore1', 'ignore2', 'state', 'ignore3', 'ignore4', 'logrecno']

acs_raw = pd.DataFrame()

for state in states:
    
    state_df = pd.DataFrame()
    
    # unzip, if needed
    if not os.path.exists(DATA_PATH + state):
            zip_filename = '{}_All_Geographies_Not_Tracts_Block_Groups.zip'.format(state)
            with zipfile.ZipFile(DATA_PATH + zip_filename, 'r') as zip_ref:
                zip_ref.extractall(DATA_PATH + state)

    # get abbreviation from file names
    data_file = os.listdir(DATA_PATH + state)
    data_file = [x for x in data_file if re.match('e20095..0001000.txt', x)]
    state_abbr = re.sub('e20095(..)0001000.txt', '\\1', data_file[0])
                
    # build lookup between logrecno and geoid
    geo_file = '{dp}{state}/g20095{st}.txt'.format(dp=DATA_PATH, state=state, st=state_abbr)
    geo_lu = pd.read_fwf(geo_file, names=GEO_COLS, widths=GEO_WIDTHS)
    geo_lu = geo_lu.loc[geo_lu['geoid'].str.match('05000US\d{5}'), ['logrecno', 'geoid', 'name']]
    geo_lu['geoid'] = geo_lu['geoid'].str.replace('^\d+US', '_')
    
    # loop through files to pull out necessary vars
    for f_raw in files_to_vars.keys():
        f_name = '{dp}{state}/{f}'.format(dp=DATA_PATH, state=state, f=f_raw.format(st=state_abbr))
        all_cols = std_cols + files_to_vars[f_raw]['header']
        temp_df = pd.read_csv(f_name, names=all_cols, encoding='latin-1', low_memory=False)
        keep_cols = ['state', 'logrecno'] + files_to_vars[f_raw]['cols']
        temp_df = temp_df[keep_cols].copy()
        if state_df.shape[0] > 0:
            state_df = pd.merge(state_df, temp_df)
        else:
            state_df = temp_df.copy()
    
    # merge to get geoid's
    state_df = pd.merge(state_df, geo_lu)
    
    acs_raw = acs_raw.append(state_df)

acs_raw = acs_raw.reset_index(drop=True)
acs_raw.to_csv('data/acs_raw.csv', index=False)
display(acs_raw.head())

,state,logrecno,B00001_001,B00002_001,B01001_001,B01001_002,B01001_003,B01001_004,B01001_005,B01001_006,...,B25035_001,B24081_001,B24081_002,B24081_005,B24081_006,B24081_007,B24081_008,B24081_009,geoid,name
0,al,13,4560,1872,49584.0,24057.0,1683.0,1824.0,2222.0,1327.0,...,1987,31559,29975,28333,35777,38866,54679,20568,_01001,"Autauga County, Alabama"
1,al,14,10722,5295,171997.0,84263.0,5543.0,5251.0,6053.0,3725.0,...,1992,28947,27796,28983,35826,36696,54868,25000,_01003,"Baldwin County, Alabama"
2,al,15,2479,1102,29663.0,15687.0,963.0,808.0,1083.0,679.0,...,1980,24864,24404,18523,30756,37561,31806,13608,_01005,"Barbour County, Alabama"
3,al,16,1348,579,21464.0,11164.0,614.0,673.0,875.0,538.0,...,1980,29348,28004,31774,27428,46250,23750,35786,_01007,"Bibb County, Alabama"
4,al,17,4389,1875,56804.0,28216.0,1857.0,1951.0,2090.0,1328.0,...,1983,29382,29299,28696,32500,32886,56087,20845,_01009,"Blount County, Alabama"


### Step 4: Build Features


In [10]:
# uncomment this cell to build the ACS features (in the following cells) without rebuilding the raw data
# acs_raw = pd.read_csv('data/acs_raw.csv')

In [11]:
acs_features_cols = ['state', 'geoid', 'name']
acs_features = acs_raw.copy()

In [12]:
# totals
acs_features['total_pop'] = acs_features['B00001_001']
acs_features['total_hholds'] = acs_features['B00002_001']
acs_features_cols.extend(['total_pop', 'total_hholds'])

# sex by age
temp_cols = ['sexbyage_m_u_18', 'sexbyage_m_18_29', 'sexbyage_m_30_39', 'sexbyage_m_40_49', 'sexbyage_m_50_64', 'sexbyage_m_65_plus', 'sexbyage_f_u_18', 'sexbyage_f_18_29', 'sexbyage_f_30_39', 'sexbyage_f_40_49', 'sexbyage_f_50_64', 'sexbyage_f_65_plus']

acs_features['sexbyage_m_u_18'] = acs_features['B01001_003'] + acs_features['B01001_004'] + acs_features['B01001_005'] + acs_features['B01001_006']
acs_features['sexbyage_m_18_29'] = acs_features['B01001_007'] + acs_features['B01001_008'] + acs_features['B01001_009'] + acs_features['B01001_010'] + acs_features['B01001_011']
acs_features['sexbyage_m_30_39'] = acs_features['B01001_012'] + acs_features['B01001_013']
acs_features['sexbyage_m_40_49'] = acs_features['B01001_014'] + acs_features['B01001_015']
acs_features['sexbyage_m_50_64'] = acs_features['B01001_016'] + acs_features['B01001_017'] + acs_features['B01001_018'] + acs_features['B01001_019']
acs_features['sexbyage_m_65_plus'] = acs_features['B01001_020'] + acs_features['B01001_021'] + acs_features['B01001_022'] + acs_features['B01001_023'] + acs_features['B01001_024'] + acs_features['B01001_025']

acs_features['sexbyage_f_u_18'] = acs_features['B01001_027'] + acs_features['B01001_028'] + acs_features['B01001_029'] + acs_features['B01001_030']
acs_features['sexbyage_f_18_29'] = acs_features['B01001_031'] + acs_features['B01001_032'] + acs_features['B01001_033'] + acs_features['B01001_034'] + acs_features['B01001_035']
acs_features['sexbyage_f_30_39'] = acs_features['B01001_036'] + acs_features['B01001_037']
acs_features['sexbyage_f_40_49'] = acs_features['B01001_038'] + acs_features['B01001_039']
acs_features['sexbyage_f_50_64'] = acs_features['B01001_040'] + acs_features['B01001_041'] + acs_features['B01001_042'] + acs_features['B01001_043']
acs_features['sexbyage_f_65_plus'] = acs_features['B01001_044'] + acs_features['B01001_045'] + acs_features['B01001_046'] + acs_features['B01001_047'] + acs_features['B01001_048'] + acs_features['B01001_049']

for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B01001_001']

# language
acs_features['speak_only_english'] = acs_features['B06007_002'] / acs_features['B06007_001']
acs_features_cols.append('speak_only_english')

# educational attainment
acs_features['educ_less_than_hs'] = acs_features['B06009_002'] / acs_features['B06009_001']
acs_features['educ_hs_grad'] = acs_features['B06009_003'] / acs_features['B06009_001']
acs_features['educ_some_college'] = acs_features['B06009_004'] / acs_features['B06009_001']
acs_features['educ_college_grad'] = acs_features['B06009_005'] / acs_features['B06009_001']
acs_features['educ_post_grad'] = acs_features['B06009_006'] / acs_features['B06009_001']
acs_features_cols.extend(['educ_less_than_hs', 'educ_hs_grad', 'educ_some_college', 'educ_college_grad', 'educ_post_grad'])

# income
acs_features['income_u10k'] = acs_features['B06010_004'] / acs_features['B06010_001']
acs_features['income_10k_to_u15k'] = acs_features['B06010_005'] / acs_features['B06010_001']
acs_features['income_15k_to_u25k'] = acs_features['B06010_006'] / acs_features['B06010_001']
acs_features['income_25k_to_u35k'] = acs_features['B06010_007'] / acs_features['B06010_001']
acs_features['income_35k_to_u50k'] = acs_features['B06010_008'] / acs_features['B06010_001']
acs_features['income_50k_to_u65k'] = acs_features['B06010_009'] / acs_features['B06010_001']
acs_features['income_65k_to_u75k'] = acs_features['B06010_010'] / acs_features['B06010_001']
acs_features['income_75k_or_more'] = acs_features['B06010_011'] / acs_features['B06010_001']
acs_features_cols.extend(['income_u10k', 'income_10k_to_u15k', 'income_15k_to_u25k', 'income_25k_to_u35k', 'income_35k_to_u50k', 'income_50k_to_u65k', 'income_65k_to_u75k', 'income_75k_or_more'])

# marital status -- excluding "married"
acs_features['marital_never_married'] = acs_features['B06008_002'] / acs_features['B06008_001']
acs_features_cols.append('marital_never_married')

# geographic mobility
bcols = [x for x in acs_features.columns if re.match('^B07401_.*', x)]
for c in bcols:
    acs_features[c] = acs_features[c].astype(str).str.replace('^\.+$', '0').astype(float)

acs_features['livinginarea_1_to_4'] = acs_features['B07401_002']
acs_features['livinginarea_5_to_17'] = acs_features['B07401_003']
acs_features['livinginarea_18_to_19'] = acs_features['B07401_004']
acs_features['livinginarea_20_to_49'] = acs_features['B07401_005'] + acs_features['B07401_006'] + acs_features['B07401_007'] + acs_features['B07401_008'] + acs_features['B07401_009'] + acs_features['B07401_010']
acs_features['livinginarea_50_plus'] = acs_features['B07401_011'] + acs_features['B07401_012'] + acs_features['B07401_013'] + acs_features['B07401_014'] + acs_features['B07401_015'] + acs_features['B07401_016']
temp_cols = [x for x in acs_features.columns if re.match('^livinginarea.*', x)]

for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B07401_001']
acs_features_cols.extend(temp_cols)

# citizenship -- excluding "citizen"
acs_features['citizenship_non_citizen'] = acs_features['B05001_006'] / acs_features['B05001_001']
acs_features_cols.append('citizenship_non_citizen')

# income vs poverty level
acs_features['incomepovertyratio_u1'] = acs_features['B05010_002']
acs_features['incomepovertyratio_1_to_2'] = acs_features['B05010_010']
acs_features['incomepovertyratio_2_plus'] = acs_features['B05010_018']
temp_cols = [x for x in acs_features.columns if re.match('^incomepovertyratio.*', x)]
for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B05010_001']
acs_features_cols.extend(temp_cols)

# household earnings -- excluding "households without earnings"
acs_features['hh_with_earnings'] = acs_features['B19051_002'] / acs_features['B19051_001']
acs_features_cols.append('hh_with_earnings')

# employment -- excluding "not in labor force"
temp_cols = ['employ_employed', 'employ_unemployed']
acs_features['employ_employed'] = acs_features['B12006_005'] + acs_features['B12006_010'] + acs_features['B12006_016'] + acs_features['B12006_021'] + acs_features['B12006_027'] + acs_features['B12006_032'] + acs_features['B12006_038'] + acs_features['B12006_043'] + acs_features['B12006_049'] + acs_features['B12006_054']
acs_features['employ_unemployed'] = acs_features['B12006_006'] + acs_features['B12006_011'] + acs_features['B12006_017'] + acs_features['B12006_022'] + acs_features['B12006_028'] + acs_features['B12006_033'] + acs_features['B12006_039'] + acs_features['B12006_044'] + acs_features['B12006_050'] + acs_features['B12006_055']

for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B12006_001']
acs_features_cols.extend(temp_cols)

# gini index of income inequality
acs_features['inequality_index_gini'] = acs_features['B19083_001']
acs_features_cols.append('inequality_index_gini')

# household type
acs_features['hhtype_married_couple'] = acs_features['B11001_003'] / acs_features['B11001_001']
acs_features['hhtype_m_householder'] = acs_features['B11001_005'] / acs_features['B11001_001']
acs_features['hhtype_f_householder'] = acs_features['B11001_006'] / acs_features['B11001_001']
acs_features['hhtype_living_alone'] = acs_features['B11001_008'] / acs_features['B11001_001']
acs_features_cols.extend(['hhtype_married_couple', 'hhtype_m_householder', 'hhtype_f_householder', 'hhtype_living_alone'])

# transportation to work -- excluding other (taxiab, motorcycle, other means)
temp_cols = ['traveltowork_drove_alone', 'traveltowork_carpooled', 'traveltowork_public_transit']
acs_features['traveltowork_drove_alone'] = acs_features['B08301_003']
acs_features['traveltowork_carpooled'] = acs_features['B08301_004']
acs_features['traveltowork_public_transit'] = acs_features['B08301_010']
acs_features['traveltowork_bike_walk'] = acs_features['B08301_018'] + acs_features['B08301_019']
acs_features['traveltowork_work_at_home'] = acs_features['B08301_021']

for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B08301_001']
acs_features_cols.extend(temp_cols)

# length of commute to work
temp_cols = ['commutetime_u10', 'commutetime_10_30', 'commutetime_30_60', 'commute_time_60_90', 'commutetime_90_plus']
acs_features['commutetime_u10'] = acs_features['B08303_002'] + acs_features['B08303_003']
acs_features['commutetime_10_30'] = acs_features['B08303_004'] + acs_features['B08303_005'] + acs_features['B08303_006'] + acs_features['B08303_007']
acs_features['commutetime_30_60'] = acs_features['B08303_008'] + acs_features['B08303_009'] + acs_features['B08303_010'] + acs_features['B08303_011']
acs_features['commute_time_60_90'] = acs_features['B08303_012']
acs_features['commutetime_90_plus'] = acs_features['B08303_013']

for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B08303_001']
acs_features_cols.extend(temp_cols)

# median year structure built
acs_features['structure_year_built_median'] = acs_features['B25035_001'].str.replace('^\.+$', '0').astype(float)
acs_features_cols.append('structure_year_built_median')

# work type -- excluding "self-employed"
bcols = [x for x in acs_features.columns if re.match('^B24081_.*', x)]
for c in bcols:
    acs_features[c] = acs_features[c].astype(str).str.replace('^\.+$', '0').astype(float)

temp_cols = ['worktype_for_profit', 'worktype_non_profit', 'worktype_government']
acs_features['worktype_for_profit'] = acs_features['B24081_002']
acs_features['worktype_non_profit'] = acs_features['B24081_005']
acs_features['worktype_government'] = acs_features['B24081_006'] + acs_features['B24081_007'] + acs_features['B24081_008']

for c in temp_cols:
    acs_features[c] = acs_features[c] / acs_features['B24081_001']
acs_features_cols.extend(temp_cols)

In [13]:
# select feature cols and output to file
acs_features = acs_features[acs_features_cols]
acs_features['state'] = acs_features['state'].str.upper()

acs_features.to_csv('data/acs_features.csv', index=False)
print(acs_features.shape)

(3221, 49)
